In [ ]:
import json

test_data_file = './testing/test_data-2.json'

def prep_data():
    with open(test_data_file) as json_file:
        data = json.load(json_file)

    contract = data['liabilities']
    return contract

In [ ]:
import spacy

nlp = spacy.load("en_core_web_md")

contracts = prep_data()
sent_list = []

for txt in contracts: 
    doc = nlp(txt['data'])
    print('Text: ', txt['data'])
    data_dict = {}
    for ent in doc.ents:        
        #print(ent.text, ent.start_char, ent.end_char, ent.label_)
        if ent.label_ == 'MONEY':
            data_dict['money'] = ent.text
        if ent.label_ == 'DATE':
            data_dict['date'] = ent.text

    for token in doc:
        if (token.pos_ == "VERB"): # and (not token.is_stop):
            data_dict['verb'] = token.text
    sent_list.append(data_dict)
    print (data_dict)

print (sent_list)

In [ ]:
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_md")

contracts = prep_data()
sent_list = []

for txt in contracts: 
    doc = nlp(txt['data'])
    print('Text: ', txt['data'])
    displacy.render(doc,jupyter=True)

    for token in doc:
        print(token.text, token.dep_, token.head.text, [child for child in token.children])

In [7]:
import spacy
nlp = spacy.load("en_core_web_md")
contracts = prep_data()

aspects = []
for txt in contracts: 
  doc = nlp(txt['data'])
  descriptive_term = ''
  target = ''
  for token in doc:
    if token.dep_ == 'nsubj' and token.pos_ == 'NOUN':
      target = token.text
    if token.pos_ == 'ADJ':
      prepend = ''
      for child in token.children:
        if child.pos_ != 'ADV':
          continue
        prepend += child.text + ' '
      descriptive_term = prepend + token.text
  aspects.append({'aspect': target,
    'description': descriptive_term})
print(aspects)

from textblob import TextBlob
for aspect in aspects:
  aspect['sentiment'] = TextBlob(aspect['description']).sentiment
print(aspects)

[{'aspect': '', 'description': ''}, {'aspect': 'company', 'description': 'short'}, {'aspect': '', 'description': 'accrued'}, {'aspect': '', 'description': 'payable'}, {'aspect': 'liabilities', 'description': 'same'}, {'aspect': '', 'description': ''}, {'aspect': 'company', 'description': 'certain'}]
[{'aspect': '', 'description': '', 'sentiment': Sentiment(polarity=0.0, subjectivity=0.0)}, {'aspect': 'company', 'description': 'short', 'sentiment': Sentiment(polarity=0.0, subjectivity=0.3)}, {'aspect': '', 'description': 'accrued', 'sentiment': Sentiment(polarity=0.0, subjectivity=0.0)}, {'aspect': '', 'description': 'payable', 'sentiment': Sentiment(polarity=0.0, subjectivity=0.0)}, {'aspect': 'liabilities', 'description': 'same', 'sentiment': Sentiment(polarity=0.0, subjectivity=0.125)}, {'aspect': '', 'description': '', 'sentiment': Sentiment(polarity=0.0, subjectivity=0.0)}, {'aspect': 'company', 'description': 'certain', 'sentiment': Sentiment(polarity=0.21428571428571427, subjecti

In [ ]:
# Transformer Based NER Model - Choosen Spacy over this. 
from transformers import AutoTokenizer, pipeline
from transformers import (
    AutoConfig,
    AutoModelForTokenClassification,
    AutoTokenizer
)

#model_path = 'cuad-models/roberta-base/'
model_path = "dslim/bert-base-NER"
#model_path = "distilbert-base-uncased"

def initialise(): 
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForTokenClassification.from_pretrained(model_path)
    question="null"
    tokenizer.encode(question, truncation=True, padding=True)

    nlp = pipeline('ner', model=model, tokenizer=tokenizer)
    return tokenizer, nlp

def predict(contract, tokenizer, nlp):
    tokenizer.encode(contract, truncation=True, padding=True)
    ner_result = nlp(contract)

    return ner_result


In [ ]:
tokenizer, nlp = initialise()
contracts = prep_data()

for txt in contracts: 
    answer = predict(txt['data'], tokenizer, nlp)
    print('Text: ', txt['data'])
    print('NER: ', answer)

In [ ]:
import spacy
import pytextrank

# example text
text = """Compatibility of systems of linear constraints over the set of natural numbers.
Criteria of compatibility of a system of linear Diophantine equations, strict inequations,
and nonstrict inequations are considered. Upper bounds for components of a minimal set of
solutions and algorithms of construction of minimal generating sets of solutions for all types
of systems are given. These criteria and the corresponding algorithms for constructing a minimal
supporting set of solutions can be used in solving all the considered types systems and systems of mixed types."""

# load a spaCy model, depending on language, scale, etc.
nlp = spacy.load("en_core_web_md")
# add PyTextRank to the spaCy pipeline
nlp.add_pipe("textrank")

doc = nlp(text)
# examine the top-ranked phrases in the document
for phrase in doc._.phrases:
    print(phrase.text)
    print(phrase.rank, phrase.count)
    print(phrase.chunks)

In [ ]:
import re
from app.PreProcessText import PreProcessText

ppt = PreProcessText()

def find_dep_key(text, phrase):
    if not text:
        return None
    text = ppt.get_lemmantizer(text)
    phrase = ppt.get_lemmantizer(phrase)
    # search phrase in text 
    found_text = re.search(phrase, text)
    if found_text:
        return found_text.group(0)    

In [4]:
#!pip3 install lexnlp 
import nltk 

import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')



[nltk_data] Downloading package punkt to
[nltk_data]     /Users/saurabhkaushik/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


[(Decimal('5.0'), 'USD'), (Decimal('1000000.0'), 'GBP')]


In [23]:
import lexnlp.extract.en.money
text = "The price will be  $4.439 billion GBP."
nlist = list(lexnlp.extract.en.money.get_money(text))
print (nlist[1][0])


9000000000.0


In [15]:
num = 545454
x = len(str(num))
print (pow(10, x))

1000000
